# Open Program - Kaan G.
In this notebook the code to beat a medium bot is present. It's all the way at the bottom of the notebook. For the rest you will see the steps i took to get there.

# Chapter 1 - Tracking legends using computer vision
Again, we can do this multiple ways. I chose the most simplistic one for now, tracking certain colors on my screen. Inside of the game *Brawlhalla* you can choose a color for your *legend*. This means we can choose a color for our legend that isn't used anywhere else inside of a *match*. this way our *legend* is the only thing being detected by the color detector.

Below you will see the slow steps I took to get to color tracking. I also listed certain sources I used per codeblock.

**Our end goal of chapter 1 is to have: A color tracker which can detect colors on my own screen, and output the locations of the bounding boxes**

## 1.0 Import Dependencies and check versions 

In [ ]:
import sys # Python 3.10.11
import cv2 # 4.6.0
import numpy as np # 1.23.4
import pyautogui as pg # 0.9.54
import PIL
from PIL import Image # 9.2.0

print(f'Python: {sys.version[0:7]}')
print(f'OpenCV2: {cv2.__version__}')
print(f'NumPy: {np.__version__}')
print(f'PyAutoGUI: {pg.__version__}')
print(f'PIL: {PIL.__version__}')

## 1.1 Capture With Laptop Camera or a Video-file and detects moving pixel-areas with an area bigger than 200px
Source: https://www.youtube.com/watch?v=GgGro5IV-cs

In [ ]:
#cap = cv2.VideoCapture("ML.mp4")
cap = cv2.VideoCapture(0)


# Object Detection from stable camera
object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40) # might wann aplay with these values for better detection

while True:
    # Get Frame
    ret, frame = cap.read()
    height, width, _ = frame.shape
    #print(ret)
    #print(frame)
    #break

    # Extract Region of interest
    roi = frame[0: 720, 0: 1280] 
    
    # Object Detection
    mask = object_detector.apply(roi)
    #_, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area > 200:
            #cv2.drawContours(roi, [cnt], -1, (0, 255, 0), 2)
            x, y, w, h, = cv2.boundingRect(cnt)
            cv2.rectangle(roi, (x, y), (x+w, y+h), (0, 255, 0), 3)
            print(x, y, w, h)
    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", mask)
    
    key = cv2.waitKey(30)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()

## 1.2 Capture With Laptop Cam and point out yellow objects
Source: https://youtu.be/aFNDh5k3SjU?si=Uc-dQ_2WLvSLEqxw

In [ ]:
def get_limits(color):
    c = np.uint8([[color]])  # BGR values
    hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)

    hue = hsvC[0][0][0]  # Get the hue value

    # Handle red hue wrap-around
    if hue >= 165:  # Upper limit for divided red hue
        lowerLimit = np.array([hue - 10, 100, 100], dtype=np.uint8)
        upperLimit = np.array([180, 255, 255], dtype=np.uint8)
    elif hue <= 15:  # Lower limit for divided red hue
        lowerLimit = np.array([0, 100, 100], dtype=np.uint8)
        upperLimit = np.array([hue + 10, 255, 255], dtype=np.uint8)
    else:
        lowerLimit = np.array([hue - 2, 100, 100], dtype=np.uint8)
        upperLimit = np.array([hue + 2, 255, 255], dtype=np.uint8)

    return lowerLimit, upperLimit


In [ ]:
yellow = [0, 255, 255]  # yellow in BGR colorspace

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    hsvImage = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lowerLimit, upperLimit = get_limits(color=yellow)

    mask = cv2.inRange(hsvImage, lowerLimit, upperLimit)
    
    mask_ = Image.fromarray(mask)

    bbox = mask_.getbbox()

    if bbox is not None:
        x1, y1, x2, y2 = bbox

        frame = cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 5)
        print(f'location: {(x1+x2)/2}, {(y1+y2)/2}')

    cv2.imshow('test', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): # quit with q
        break

cap.release()

cv2.destroyAllWindows()


## 1.3 Base code to Capture Screen
Finally realised that video capturing screen is nothing different from making screenshots really quick

In [ ]:
while True:
    # get frame
    screenshot = cv2.cvtColor(np.array(pg.screenshot()), cv2.COLOR_RGB2BGR)
    print(screenshot)
    
    # show frame
    cv2.imshow('Screenshot', screenshot)
    
    # wait 1ms for a key press (for if you want to quit the program)
    if cv2.waitKey(1) & 0xFF == 27: break # press escape-key to break

cv2.destroyAllWindows()

## 1.4 Capture screen and detect pixels
Now I can start trying to combine the multiple techniques I have learned

In [ ]:
# Define Object Detector
object_detector = cv2.createBackgroundSubtractorMOG2(
    history=100, varThreshold=40)

while True:
    # get frame
    screenshot = cv2.cvtColor(np.array(pg.screenshot()), cv2.COLOR_RGB2BGR)

    # Conversion from Screen-reading-code to the other piece of code 
    frame = screenshot

    # read out frame
    height, width, _ = frame.shape

    # Extract Region of interest
    roi = frame[0: 720, 0: 1280]

    # Object Detection
    mask = object_detector.apply(roi)

    # Find contours (what are contours 😭)
    contours, _ = cv2.findContours(
        mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Calculate area and remove small elements
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 200:
            #cv2.drawContours(roi, [cnt], -1, (0, 255, 0), 2)
            x, y, w, h, = cv2.boundingRect(cnt)
            cv2.rectangle(roi, (x, y), (x+w, y+h), (0, 255, 0), 3)
            print(x, y, w, h)



    # show frame
    cv2.imshow('Screen Capture - Detect Moving Pixels', screenshot)
    #cv2.imshow("Mask", mask)

    # show for a duration of 1ms
    if cv2.waitKey(1) & 0xFF == 27: break  # press escape-key to break

cv2.destroyAllWindows()

## 1.5 Capture Screen and Track multiple colors
Source: https://www.youtube.com/watch?v=ddSo8Nb0mTw&t=290s

In [ ]:
# Start a while loop
while(True):

    screenshot = cv2.cvtColor(np.array(pg.screenshot()), cv2.COLOR_RGB2BGR)
    frame = screenshot
    # Convert the imageFrame in
    # BGR(RGB color space) to
    # HSV(hue-saturation-value)
    # color space
    hsvFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Set range for red color and
    # define mask
    red_lower = np.array([136, 87, 111], np.uint8)
    red_upper = np.array([180, 255, 255], np.uint8)
    red_mask = cv2.inRange(hsvFrame, red_lower, red_upper)

    # Set range for green color and
    # define mask
    green_lower = np.array([25, 52, 72], np.uint8)
    green_upper = np.array([102, 255, 255], np.uint8)
    green_mask = cv2.inRange(hsvFrame, green_lower, green_upper)

    # Set range for blue color and
    # define mask
    blue_lower = np.array([94, 80, 2], np.uint8)
    blue_upper = np.array([120, 255, 255], np.uint8)
    blue_mask = cv2.inRange(hsvFrame, blue_lower, blue_upper)

    # Morphological Transform, Dilation
    # for each color and bitwise_and operator
    # between imageFrame and mask determines
    # to detect only that particular color
    kernel = np.ones((5, 5), "uint8")

    # For red color
    red_mask = cv2.dilate(red_mask, kernel)
    res_red = cv2.bitwise_and(frame, frame,
                              mask=red_mask)

    # For green color
    green_mask = cv2.dilate(green_mask, kernel)
    res_green = cv2.bitwise_and(frame, frame,
                                mask=green_mask)

    # For blue color
    blue_mask = cv2.dilate(blue_mask, kernel)
    res_blue = cv2.bitwise_and(frame, frame,
                               mask=blue_mask)

    # Creating contour to track red color
    contours, hierarchy = cv2.findContours(red_mask,
                                           cv2.RETR_TREE,
                                           cv2.CHAIN_APPROX_SIMPLE)

    for pic, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if(area > 300):
            x, y, w, h = cv2.boundingRect(contour)
            frame = cv2.rectangle(frame, (x, y),
                                       (x + w, y + h),
                                       (0, 0, 255), 2)

            cv2.putText(frame, "Red Colour", (x, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0,
                        (0, 0, 255))

    # Creating contour to track green color
    contours, hierarchy = cv2.findContours(green_mask,
                                           cv2.RETR_TREE,
                                           cv2.CHAIN_APPROX_SIMPLE)

    for pic, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if(area > 300):
            x, y, w, h = cv2.boundingRect(contour)
            frame = cv2.rectangle(frame, (x, y),
                                       (x + w, y + h),
                                       (0, 255, 0), 2)

            cv2.putText(frame, "Green Colour", (x, y),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1.0, (0, 255, 0))

    # Creating contour to track blue color
    contours, hierarchy = cv2.findContours(blue_mask,
                                           cv2.RETR_TREE,
                                           cv2.CHAIN_APPROX_SIMPLE)
    for pic, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if(area > 300):
            x, y, w, h = cv2.boundingRect(contour)
            frame = cv2.rectangle(frame, (x, y),
                                       (x + w, y + h),
                                       (255, 0, 0), 2)

            cv2.putText(frame, "Blue Colour", (x, y),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1.0, (255, 0, 0))

    # Program Termination
    cv2.imshow("Multiple Color Detection in Real-TIme", frame)
    if cv2.waitKey(10) & 0xFF == 27:
        break

cv2.destroyAllWindows()


## 1.6 Capture Screen and Detect Jake

In [ ]:
import numpy as np # 1.23.4
import pyautogui as pg  # 0.9.54
import cv2 # 4.6.0.66


# Start a while loop
while(True):

    screenshot = cv2.cvtColor(np.array(pg.screenshot()), cv2.COLOR_RGB2BGR)
    frame = screenshot
    # Convert the imageFrame in
    # BGR(RGB color space) to
    # HSV(hue-saturation-value)
    # color space
    hsvFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Set range for red color and
    # define mask
    red_lower = np.array([136, 87, 111], np.uint8)
    red_upper = np.array([180, 255, 255], np.uint8)
    # jake colors
    red_upper = np.array([45, 190, 255], np.uint8)  # B G R
    red_lower = np.array([0, 160, 200], np.uint8)
    # jake colors
    red_upper = np.array([45, 255, 255], np.uint8)  # B G R
    red_lower = np.array([0, 175, 200], np.uint8)

    red_mask = cv2.inRange(hsvFrame, red_lower, red_upper)

    # Morphological Transform, Dilation
    # for each color and bitwise_and operator
    # between imageFrame and mask determines
    # to detect only that particular color
    kernel = np.ones((5, 5), "uint8")

    # For red color
    red_mask = cv2.dilate(red_mask, kernel)
    res_red = cv2.bitwise_and(frame, frame,
                              mask=red_mask)

    # Creating contour to track red color
    contours, hierarchy = cv2.findContours(red_mask,
                                           cv2.RETR_TREE,
                                           cv2.CHAIN_APPROX_SIMPLE)

    for pic, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if(area > 300):
            x, y, w, h = cv2.boundingRect(contour)
            frame = cv2.rectangle(frame, (x, y),
                                  (x + w, y + h),
                                  (0, 0, 255), 2)

            cv2.putText(frame, "Jake Base", (x, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0,
                        (0, 0, 255))
            
            print(f'x: {(x+w)/2}, y: {(y+h)/2}') # Extracted Game Variables
        else:
            print(f'no Jake Base')


    # Program Termination
    cv2.imshow("Jake Detector 2 Pro++ Official", frame)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cv2.destroyAllWindows()


## 1.7 Capture Screen and Detect Multiple Jakes

In [ ]:
import numpy as np  # 1.23.4
import pyautogui as pg  # 0.9.54
import cv2  # 4.6.0.66

min_area = 2000 # minimum area for pixel-color area to be detected

# Start a while loop
while(True):

    screenshot = cv2.cvtColor(np.array(pg.screenshot()), cv2.COLOR_RGB2BGR)
    frame = screenshot
    # Convert the imageFrame in
    # BGR(RGB color space) to
    # HSV(hue-saturation-value)
    # color space
    hsvFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Set range for red color and
    # Red Jake
    red_lower = np.array([136, 87, 111], np.uint8)
    red_upper = np.array([180, 255, 255], np.uint8)
    red_mask = cv2.inRange(hsvFrame, red_lower, red_upper)
    # Base Jake (Lightning Yellow)
    jake_upper = np.array([45, 255, 255], np.uint8)  # B G R
    jake_lower = np.array([0, 175, 200], np.uint8)
    jake_mask = cv2.inRange(hsvFrame, jake_lower, jake_upper)
    # Blue Jake
    blue_lower = np.array([50, 190, 190], np.uint8)
    blue_upper = np.array([255, 230, 230], np.uint8)
    blue_mask = cv2.inRange(hsvFrame, blue_lower, blue_upper)



    # Morphological Transform, Dilation
    # for each color and bitwise_and operator
    # between imageFrame and mask determines
    # to detect only that particular color
    kernel = np.ones((5, 5), "uint8")

    # For red color
    jake_mask = cv2.dilate(jake_mask, kernel)
    res_jake = cv2.bitwise_and(frame, frame,
                              mask=jake_mask)

    # # Creating contour to track base jake color
    # contours, hierarchy = cv2.findContours(jake_mask,
    #                                        cv2.RETR_TREE,
    #                                        cv2.CHAIN_APPROX_SIMPLE)

    # for pic, contour in enumerate(contours):
    #     area = cv2.contourArea(contour)
    #     if(area > min_area):
    #         x, y, w, h = cv2.boundingRect(contour)
    #         frame = cv2.rectangle(frame, (x, y),
    #                               (x + w, y + h),
    #                               (0, 0, 255), 2)

    #         cv2.putText(frame, "Jake Base", (x, y),
    #                     cv2.FONT_HERSHEY_SIMPLEX, 1.0,
    #                     (0, 0, 255))

    #         print(f'Jake Base - x: {(x+w)/2}, y: {(y+h)/2}')  # Extracted Game Variables
    #     else:
    #         print(f'no Jake Base')

    # For red color
    red_mask = cv2.dilate(red_mask, kernel)
    res_red = cv2.bitwise_and(frame, frame,
                              mask=red_mask)

    # Creating contour to track red color
    contours, hierarchy = cv2.findContours(red_mask,
                                           cv2.RETR_TREE,
                                           cv2.CHAIN_APPROX_SIMPLE)

    for pic, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if(area > min_area):
            x, y, w, h = cv2.boundingRect(contour)
            frame = cv2.rectangle(frame, (x, y),
                                  (x + w, y + h),
                                  (0, 0, 255), 2)

            cv2.putText(frame, "Jake Red", (x, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0,
                        (0, 0, 255))

            print(f'Jake Red - x: {(x+w)/2}, y: {(y+h)/2}')  # Extracted Game Variables
        else:
            print(f'no Jake Red')


    # Creating contour to track blue color
    contours, hierarchy = cv2.findContours(blue_mask,
                                           cv2.RETR_TREE,
                                           cv2.CHAIN_APPROX_SIMPLE)
    for pic, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if(area > 300):
            x, y, w, h = cv2.boundingRect(contour)
            frame = cv2.rectangle(frame, (x, y),
                                  (x + w, y + h),
                                  (255, 0, 0), 2)

            cv2.putText(frame, "Blue Colour", (x, y),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1.0, (255, 0, 0))
            print(f'Jake Blue - x: {(x+w)/2}, y: {(y+h)/2}')
        else:
            print(f'no Jake Blue')

    # Program Termination
    cv2.imshow("Jake Detector 2 Pro++ Official", frame)
    if cv2.waitKey(5) & 0xFF == 27:
        break
cv2.destroyAllWindows()


In [ ]:
!pip install pygetwindow

## 1.9 Controlling Red "Orion For Hire" Orion

In [4]:
import numpy as np  # 1.23.4
import pyautogui as pg  # 0.9.54
import cv2  # 4.6.0.66
import time
import pygetwindow as gw
import win32gui
import win32con
import random as rnd
from modules.movement import move_left_and_attack, move_right_and_attack, jump_to_stage_and_recovery, dsig

red_lower = np.array([136, 87, 111], np.uint8) # min red color
red_upper = np.array([180, 255, 255], np.uint8) # max red color
left = 1080 # left vertical line - decide when to move right or not
right = 1300 # right vertical line - decide when to move left or not
bottom = 424 # bottom horizontal line - decide when to jump or not
top = 300 # top horizontal line - when below top line you can start using dsigs

_DOWN_BUTTON = 's'
_JUMP_BUTTON = 'space'
_LIGHT_BUTTON = 'j'
_HEAVY_BUTTON = 'k'
_LEFT_BUTTON = 'a'
_RIGHT_BUTTON = 'd'

time.sleep(2)
# pg.press(_JUMP_BUTTON)

m=0

# Start Bot
while(True):
    m+=1

    # Take Screenshot
    frame = cv2.cvtColor(np.array(pg.screenshot()), cv2.COLOR_RGB2BGR)
    
    # convert to hsv for color detecting later
    hsvFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # define mask?
    red_mask = cv2.inRange(hsvFrame, red_lower, red_upper)

    # Creating contour to track Red color
    contours, hierarchy = cv2.findContours(red_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # try to get rid of for loop
    for contour in contours:
        area = cv2.contourArea(contour)
        if(area >= 200):
            x, y, w, h = cv2.boundingRect(contour)
            frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)            
            # makes it play as weird as Xefi
            pg.keyUp(_DOWN_BUTTON) # reset holding down
            if y >= bottom: # jump if too low
                jump_to_stage_and_recovery(x, y, left, right)
            elif x <= left:
                move_right_and_attack(x, y)
            elif x >= right:
                move_left_and_attack(x, y)
            elif y < bottom and y > top: # centered and close to ground
                dsig(x, y)

    # visualisation (makes app lag)
    # cv2.imshow("Jake Detector 2 Pro++ Official - (Now also detecting Red Onions)", frame)
    if m == 1000000:
        break
print(m)
cv2.destroyAllWindows()


R 719 72
R 690 34
R 1061 368
R 770 112
R 745 74
R 810 249
R 783 101
R 1012 391
R 1056 370
R 976 394
dsig 1115 381
R 974 371
L 1316 393
L 1314 287
dsig 1213 398
R 1016 131
R 1010 344
R 980 380
dsig 1169 383
dsig 1182 382
R 1047 394
dsig 1187 321
dsig 1092 321
dsig 1214 375
dsig 1199 388
dsig 1170 305
dsig 1191 323
dsig 1193 386
dsig 1096 313
dsig 1099 344
dsig 1104 386
dsig 1088 392
dsig 1100 383
dsig 1088 392
dsig 1191 303
J 770 561
J 779 612
J 779 453
R 904 178
R 936 165
R 1025 212
L 1436 370
J 1539 465
J 1486 697
J 1435 697
J 1490 621
L 1512 372
L 1507 306
L 1443 227
L 1435 376
L 1437 334
R 808 219
J 1547 505
R 1025 331
J 503 872
R 701 408
R 710 364
J 201 568
J 215 535
R 794 161
R 812 116
dsig 1139 380
L 1368 393
L 1302 370
dsig 1243 380
dsig 1162 330
dsig 1157 383
R 993 250
dsig 1197 380
L 1409 395
L 1321 371
dsig 1210 371
dsig 1187 329
dsig 1177 388
dsig 1194 392
dsig 1177 388
dsig 1194 392
dsig 1268 327
dsig 1262 381
L 1321 294
dsig 1264 315
R 1033 35
J 774 433
J 779 441
R 1002 33

KeyboardInterrupt: 

In [ ]:
#14.5 met imshow
# 14.5s voor 200 frames
# 1.45s voor 20 frames

# 14.3s zonder imshow en frame
# 1.43s voor 20 frames

# 13.1s met wait key op 1 ipv 5
# 1.31s voor 20 framesj

# 11s zonder wait key
# 1.1s voor 20 frames

# 12.2s